In [66]:
%matplotlib ipympl

from mpl_toolkits.mplot3d import Axes3D
import itk
import itkwidgets
import pykonal
import seispy

In [2]:
ak135 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/AK135F_AVG.csv'),
    header=None,
    names=('depth', 'density', 'vp', 'vs', 'Q_kappa', 'Q_mu')
)
ak135['radius'] = 6371 - ak135['depth']
ak135 = ak135.sort_values('radius')

iasp91 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/IASP91.csv'),
    header=None,
    names=('depth', 'radius', 'vp', 'vs')
)

mit2008 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/mitp2008.xzv'),
    header=None,
    names=('degrees', 'radius', 'dv'),
    delim_whitespace=True
)
mit2008['radians'] = np.pi - np.radians(mit2008['degrees'] + 16.5)
mit2008 = mit2008.groupby(['radians', 'radius']).mean().reset_index()

In [ ]:
src_idx                        = (600, 0, 0)
far_field                      = pykonal.EikonalSolver(coord_sys='spherical')
far_field.vgrid.min_coords     = 10, np.pi/2, np.pi/2
far_field.vgrid.node_intervals = 10, 1, np.pi/20
far_field.vgrid.npts           = 638, 1, 40
far_field.vv                   = np.ones(far_field.vgrid.npts)

for ir in range(far_field.vgrid.npts[0]):
    far_field.vv[ir] = np.interp(far_field.vgrid[ir,0,0,0], ak135['radius'], ak135['vp'])

far_field.uu[src_idx]     = 0
far_field.is_far[src_idx] = False
far_field.close.push(*src_idx)
%time far_field.solve()


pgrid = far_field.pgrid[...]
vgrid = far_field.vgrid[...]
uu = far_field.uu
pgrid = np.append(pgrid, pgrid[:,:,0].reshape(*pgrid.shape[:2], 1, 3), axis=2)
vgrid = np.append(vgrid, vgrid[:,:,0].reshape(*vgrid.shape[:2], 1, 3), axis=2)
uu   = np.append(far_field.uu, far_field.uu[...,0].reshape(*far_field.uu.shape[:2], 1), axis=2)
vv   = np.append(far_field.vv, far_field.vv[...,0].reshape(*far_field.vv.shape[:2], 1), axis=2)
xxp  = pgrid[...,0] * np.sin(pgrid[...,1]) * np.cos(pgrid[...,2])
yyp  = pgrid[...,0] * np.sin(pgrid[...,1]) * np.sin(pgrid[...,2])
xxv  = vgrid[...,0] * np.sin(vgrid[...,1]) * np.cos(vgrid[...,2])
yyv  = vgrid[...,0] * np.sin(vgrid[...,1]) * np.sin(vgrid[...,2])


plt.close('all')
fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1, aspect=1)
qmesh = ax1.pcolormesh(
    xxv[:,0,:],
    yyv[:,0,:],
    vv[:,0,:],
    cmap=plt.get_cmap('hot'),
    shading='gouraud'
)
cbar = fig.colorbar(qmesh, ax=ax1, orientation='horizontal')
cbar.set_label('Velocity [km/s]')
ax1.scatter(
    xxv[src_idx],
    yyv[src_idx],
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)
ax2 = fig.add_subplot(1, 2, 2, aspect=1)
qmesh = ax2.pcolormesh(
    xxp[:,0,:],
    yyp[:,0,:],
    uu[:,0,:],
    cmap=plt.get_cmap('hot_r'),
    shading='gouraud',
)
ax2.yaxis.tick_right()
cbar = fig.colorbar(qmesh, ax=ax2, orientation='horizontal')
cbar.set_label('Travel time [s]')
ax2.scatter(
    xxp[src_idx],
    yyp[src_idx],
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)

In [ ]:
plt.savefig('../figures/global_slice.png', bbox_inches='tight')
plt.savefig('../figures/global_slice.pdf', bbox_inches='tight')

In [3]:
lon_min, lon_max, n_lon = -125, -70, 111
lat_min, lat_max, n_lat = 25, 49, 49
z_min, z_max, n_z       = 0, 440, 45


rho_min, rho_max, n_rho       = 6371. - z_max, 6371. - z_min, n_z
theta_min, theta_max, n_theta = np.pi/2 - np.radians(lat_max), np.pi/2 - np.radians(lat_min), n_lat
phi_min, phi_max, n_phi       = np.radians(lon_min), np.radians(lon_max), n_lon

d_rho   = (rho_max - rho_min)     / (n_rho - 1)
d_theta = (theta_max - theta_min) / (n_theta - 1)
d_phi   = (phi_max - phi_min)     / (n_phi - 1)

In [74]:
ir, it, ip                     = slice(n_rho), 9, slice(n_phi)
src_idx                        = (n_rho-2, 9, 20)
far_field                      = pykonal.EikonalSolver(coord_sys='spherical')
far_field.vgrid.min_coords     = rho_min, theta_min, theta_max
far_field.vgrid.node_intervals = d_rho, d_theta, d_phi
far_field.vgrid.npts           = n_rho, n_theta, n_phi
far_field.vv                   = np.ones(far_field.vgrid.npts)

for _ir in range(far_field.vgrid.npts[0]):
    far_field.vv[_ir] = np.interp(far_field.vgrid[_ir,0,0,0], ak135['radius'], ak135['vp'])

far_field.uu[src_idx]     = 0
far_field.is_far[src_idx] = False
far_field.close.push(*src_idx)
%time far_field.solve()


pgrid = far_field.pgrid[...]
vgrid = far_field.vgrid[...]
uu   = far_field.uu
vv   = far_field.vv
xxp  = pgrid[...,0] * np.sin(pgrid[...,1]) * np.cos(pgrid[...,2])
yyp  = pgrid[...,0] * np.sin(pgrid[...,1]) * np.sin(pgrid[...,2])
zzp  = pgrid[...,0] * np.cos(pgrid[...,1])
xxv  = vgrid[...,0] * np.sin(vgrid[...,1]) * np.cos(vgrid[...,2])
yyv  = vgrid[...,0] * np.sin(vgrid[...,1]) * np.sin(vgrid[...,2])
zzv  = vgrid[...,0] * np.cos(vgrid[...,1])

plt.close('all')
fig = plt.figure(figsize=(9, 9))
ax1 = fig.add_subplot(2, 1, 1, aspect=1)
qmesh = ax1.pcolormesh(
    xxv[ir, it, ip],
    yyv[ir, it, ip],
    vv[ir, it, ip],
    cmap=plt.get_cmap('hot'),
    shading='gouraud'
)
cbar = fig.colorbar(qmesh, ax=ax1, orientation='horizontal')
cbar.set_label('Velocity [km/s]')
ax1.scatter(
    xxv[src_idx],
    yyv[src_idx],
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)
ax2 = fig.add_subplot(2, 1, 2, aspect=1)
qmesh = ax2.pcolormesh(
    xxp[ir, it, ip],
    yyp[ir, it, ip],
    uu[ir, it, ip],
    cmap=plt.get_cmap('hot_r'),
    shading='gouraud',
)
ax2.yaxis.tick_right()
cbar = fig.colorbar(qmesh, ax=ax2, orientation='horizontal')
cbar.set_label('Travel time [s]')
ax2.scatter(
    xxp[src_idx],
    yyp[src_idx],
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)

CPU times: user 1min 3s, sys: 41.6 ms, total: 1min 3s
Wall time: 1min 3s


FigureCanvasNbAgg()

In [80]:
pgrid = seispy.coords.as_spherical(far_field.pgrid[...]).to_cartesian()
vgrid = seispy.coords.as_spherical(far_field.vgrid[...]).to_cartesian()
uu   = far_field.uu
vv   = far_field.vv

plt.close('all')
fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(1, 1, 1, projection='3d')
pts = ax.scatter(
    pgrid[ir, it, ip, 0].flatten(),
    pgrid[ir, it, ip, 1].flatten(),
    pgrid[ir, it, ip, 2].flatten(),
    c=uu[ir, it, ip].flatten(),
    cmap=plt.get_cmap('hot_r')
)
cbar = fig.colorbar(qmesh, ax=ax, orientation='horizontal')
cbar.set_label('Travel time [s]')
#ax2.scatter(
#    pgrid[ir, it, ip, 0],
#    pgrid[ir, it, ip, 1],
#    s=250,
#    marker='*',
#    edgecolor='k',
#    facecolor='w',
#    linewidth=1
#)

FigureCanvasNbAgg()

In [6]:
uui = pykonal.LinearInterpolator3D(far_field.pgrid, far_field.uu)

In [7]:
rtp.shape

(107, 49, 47, 3)

In [8]:
uu = np.full(rtp.shape[:-1], fill_value=np.inf)
for ir in range(rtp.shape[0]):
    for it in range(rtp.shape[1]):
        for ip in range(rtp.shape[2]):
            try:
                uu[ir, it, ip] = uui(rtp[ir, it, ip])
            except:
                continue

In [9]:
import itk
import itkwidgets

In [20]:
uu[np.isinf(uu)] = np.nan
img = itk.GetImageFromArray(uu)

In [27]:
itkwidgets.view(img)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageD3; proxy of <Swig Object of type 'itkIma…

In [28]:
img.SetSpacing([2, 1, 1])

In [65]:
itk.Transform.F2?

Init signature: itk.Transform.F2(*args, **kwargs)
Docstring:     
Transform points and vectors from an input space to an output space.

This abstract class defines the generic interface for a geometric
transformation from one space to another. The class provides methods
for mapping points, vectors and covariant vectors from the input space
to the output space.

Given that transformations are not necessarily invertible, this basic
class does not provide the methods for back transformation. Back
transform methods are implemented in derived classes where
appropriate.

Registration Framework Support Typically a Transform class has several
methods for setting its parameters. For use in the registration
framework, the parameters must also be represented by an array of
doubles to allow communication with generic optimizers. The Array of
transformation parameters is set using the SetParameters() method.
Another requirement of the registration framework is the computation
of the transform Jacob

In [47]:
Pi:q
    xelType = itk.UC
PixelType = itk.ctype("unsigned char")
Dimension = 3
#
MeshTraits = itk.DefaultStaticMeshTraits[PixelType, Dimension, Dimension]
PointSetType = itk.PointSet[PixelType, Dimension, MeshTraits]
#PointSet = PointSetType.New()
#
#points = PointSet.GetPoints()
#
## Create points
#p0 = itk.Point[PixelType, Dimension]()
#p1 = itk.Point[PixelType, Dimension]()
#p2 = itk.Point[PixelType, Dimension]()
#
#p0[0] = 0.0
#p0[1] = 0.0
#p0[2] = 0.0
#p1[0] = 0.1
#p1[1] = 0.0
#p1[2] = 0.0
#p2[0] = 0.0
#p2[1] = 0.1
#p2[2] = 0.0
#
#points.InsertElement(0, p0)
#points.InsertElement(1, p1)
#points.InsertElement(2, p2)

TemplateTypeError: itk.DefaultStaticMeshTraits is not wrapped for input type `itk.UC, int, int`.

To limit the size of the package, only a limited number of
types are available in ITK Python. To print the supported
types, run the following command in your python environment:

    itk.DefaultStaticMeshTraits.GetTypes()

Possible solutions:
* If you are an application user:
** Convert your input image into a supported format (see below).
** Contact developer to report the issue.
* If you are an application developer, force input images to be
loaded in a supported pixel type.

    e.g.: instance = itk.DefaultStaticMeshTraits[itk.SS, int].New(my_input)

* (Advanced) If you are an application developer, build ITK Python yourself and
turned to `ON` the corresponding CMake option to wrap the pixel type or image
dimension you need. When configuring ITK with CMake, you can set
`ITK_WRAP_${type}` (replace ${type} with appropriate pixel type such as
`double`). If you need to support images with 4

In [33]:
itk.DefaultStaticMeshTraits.GetTypes()

<itkTemplate itk::DefaultStaticMeshTraits>
Options:
  [<class 'itkMatrixPython.itkMatrixD22'>, 2, 2, <itkCType float>, <itkCType float>, <class 'itkMatrixPython.itkMatrixD22'>]
  [<class 'itkMatrixPython.itkMatrixD33'>, 3, 3, <itkCType float>, <itkCType float>, <class 'itkMatrixPython.itkMatrixD33'>]
  [<class 'itkVectorPython.itkVectorF2'>, 2, 2, <itkCType float>, <itkCType float>, <class 'itkVectorPython.itkVectorF2'>]
  [<class 'itkVectorPython.itkVectorF3'>, 3, 3, <itkCType float>, <itkCType float>, <class 'itkVectorPython.itkVectorF3'>]
  [<itkCType double>, 2]
  [<itkCType double>, 2, 2, <itkCType double>, <itkCType double>, <itkCType double>]
  [<itkCType double>, 2, 2, <itkCType float>, <itkCType float>, <itkCType double>]
  [<itkCType double>, 3]
  [<itkCType double>, 3, 3, <itkCType double>, <itkCType double>, <itkCType double>]
  [<itkCType double>, 3, 3, <itkCType float>, <itkCType float>, <itkCType double>]
  [<itkCType float>, 2]
  [<itkCType float>, 3]
  [<itkCType signe